In [1]:
import sklearn
sklearn.__version__

'1.2.2'

In [2]:
%pip install  python3
%pip install  tensorflow
%pip install  scikit-learn
%pip install  pandas
%pip install  matplotlib
%pip install  numpy
%pip install seaborn
%pip install catboost
%pip install sagemaker
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement python3 (from versions: none)
ERROR: No matching distribution found for python3
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pac

In [3]:
%pip install sagemaker

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 1. Initialize Boto3 SDK and create S3 bucket.

In [4]:
from pandas.compat import F
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler , LabelEncoder
#from tensorflow import keras
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score

ImportError: cannot import name 'F' from 'pandas.compat' (/home/humbu/.local/lib/python3.10/site-packages/pandas/compat/__init__.py)

In [ ]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'deployement-sagemaker-bucket' # Mention the created S3 bucket name here
print("Using bucket " + bucket)

3. Data Exploration and Understanding

In [ ]:
# creation de variable   qui vont  recuperer les chemins des  differentes  ensemble de  donnee

file_incart_path = 'Downloads/aws-sage-maker/INCART 2-lead Arrhythmia Database.csv'
file_sudden_path ='Downloads/aws-sage-maker/MIT-BIH Arrhythmia Database.csv'
file_mit_supra_path = 'Downloads/aws-sage-maker/MIT-BIH Supraventricular Arrhythmia Database.csv'
file_mit_bih_path = 'Downloads/aws-sage-maker/Sudden Cardiac Death Holter Database.csv'

In [ ]:
incart =  pd.read_csv(file_incart_path , low_memory= False )
sudden =  pd.read_csv(file_sudden_path , low_memory= False)
supra =  pd.read_csv(file_mit_supra_path , low_memory= False)
mitbih = pd.read_csv(file_mit_bih_path  , low_memory= False )

In [ ]:
incart.head()

In [ ]:
# Combine dataframes in the list into one dataframe

liste_des_ensemble_de_donnees  = [incart, sudden, mitbih, supra]
combinaison_df = pd.concat(liste_des_ensemble_de_donnees, ignore_index=True)

In [ ]:
combinaison_df.to_csv('donnee_carythma.csv', index=False)


In [ ]:
def netoyage_combinaison_melange_des_donnees(dataframes_list):

    combinaison_df = pd.concat(dataframes_list, ignore_index=True )
    combinaison_df = combinaison_df.dropna()
    combinaison_df = combinaison_df.reset_index(drop=True)
    melanger_df = combinaison_df.sample(frac=1, random_state=42)

    return melanger_df


#liste_des_ensemble_de_donnees  = dfcarythma # [incart, sudden, mitbih, supra]


combiner_netoyer_melanger  = netoyage_combinaison_melange_des_donnees(liste_des_ensemble_de_donnees)


combiner_netoyer_melanger.to_csv('combine.csv')
#combined_df = pd.concat([df1, df2, df3, df4], axis=0)

# Save the concatenated DataFrame to a CSV file
#combined_df.to_csv('combined_data.csv', index=False)


In [ ]:
combiner_netoyer_melanger.columns

In [ ]:

def selection_et_renaming_des_colonnes(donnee_entrer, colonnes_selectionner, nouveau_nom_de_colonnes):
    # dans ce cas  nous allons   selectonner  des  colonnes specifique s
    ensemble_de_donnee_selectionner = donnee_entrer[colonnes_selectionner].copy()

    # Renomage des   colonnes  selectionner
    ensemble_de_donnee_selectionner.columns = nouveau_nom_de_colonnes

    return ensemble_de_donnee_selectionner



# list des colonnes a  selectionner
colonnes_select= ['type', '0_pre-RR', '0_pq_interval', '0_qt_interval']

nouveau_nom_col = ['categorie arythmie', 'Interval-RR', 'Interval-PR', 'Interval-QT']

ensemble_de_donnee_selection_et_renommer = selection_et_renaming_des_colonnes(combiner_netoyer_melanger, colonnes_select, nouveau_nom_col)

ensemble_de_donnee_selection_et_renommer


In [ ]:
print(combiner_netoyer_melanger.describe)



In [ ]:
ensemble_de_donnee_selection_et_renommer

In [ ]:
import pandas as pd
import math

def calcule_du_rythme_cardiaque(rr_intervale_en_ms):
    rythme_cardiaque = 60000/ rr_intervale_en_ms
    return rythme_cardiaque

def ratioprqt(pr_intervale_en_ms, qt_intervale_en_ms):
    pr_qt_ratio = pr_intervale_en_ms / qt_intervale_en_ms
    return pr_qt_ratio

def calcule_du_qtc(qt_intervale_en_ms, rythme_cardiaque):
    qtc = qt_intervale_en_ms + 1.75 * (rythme_cardiaque - 60)
    return qtc

def creation_de_features(donnee_entrer):
    donnee_entrer['frequence cardiaque'] = donnee_entrer['Interval-RR'].apply(calcule_du_rythme_cardiaque)
    donnee_entrer['RatioPRQT'] = donnee_entrer.apply(lambda row: ratioprqt(row['Interval-PR'], row['Interval-QT']), axis=1)
    donnee_entrer['QTc'] = donnee_entrer.apply(lambda row: calcule_du_qtc(row['Interval-QT'], row['frequence cardiaque']), axis=1)

    return donnee_entrer

# nous allons  utiliser les  function  si dessus  pour les faire  les differen
# calcule concernant le  heart rate, PR/QT ratio, QTc, et creer les features
donnee_traitee = creation_de_features(ensemble_de_donnee_selection_et_renommer)


donnee_traitee

In [ ]:
donnee_traitee['categorie arythmie'].unique()

In [ ]:
donnee_traitee

In [ ]:
selected_features = [ 'Interval-RR', 'Interval-PR','Interval-QT','frequence cardiaque', 'RatioPRQT', 'QTc']

# sous  diviser les ensemble de  donnee  se basant  sur  les   features selectionner
selected_features_df = donnee_traitee[selected_features]

sns.set(style="ticks")
sns.pairplot(selected_features_df)
plt.show()

In [ ]:
def split_data(donnee_entrer , colonne_rechercher, test_size=0.2, random_state=None):

    X = donnee_entrer.drop(columns=[colonne_rechercher])
    y = donnee_entrer[colonne_rechercher]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [ ]:
def echellonement_numerique(donnee_entrer, colonnes_numeriques):

    scaler = StandardScaler()
    donnee_entrer[colonnes_numeriques] = scaler.fit_transform(donnee_entrer[colonnes_numeriques])
    return donnee_entrer

In [ ]:
colonne_rechercher = 'categorie arythmie'


#colonnes_cat = ['categorie arythmie']
colonne_num = ['Interval-RR', 'Interval-PR', 'Interval-QT','frequence cardiaque', 'RatioPRQT', 'QTc']

donne_encoder = echellonement_numerique(donnee_traitee.copy(), colonne_num)


#donnee_echelloner = echellonement_numerique(donne_encoder.copy(), colonne_num)

X_train, X_test, y_train, y_test = split_data(donne_encoder, colonne_rechercher, test_size=0.2 , random_state=42)

In [ ]:
X_train, X_test, y_train, y_test

In [ ]:
###   cette partie consiste a renommer les information  categoritielle de la sante du patient se base sur les donnees ecg 
# Your original categorical data
original_data = ['N', 'VEB', 'SVEB', 'F', 'Q']

# Define your mapping
mapping = {
    'N': 'Normale',
    'VEB': 'Battement ectopique ventriculaire  , pas  bon ',
    'SVEB': 'Battement ectopique supraventriculaire ,  pas  bon ',
    'F': 'Battement de fusion , pas bon ',
    'Q': 'Battement inconnu , pas  bon '
}

# Create a function to apply the mapping to an array


y_train = y_train.map(mapping).fillna(y_train)
y_test = y_test.map(mapping).fillna(y_test)
X_train = y_train.map(mapping).fillna(X_train)
X_test = y_test.map(mapping).fillna(X_test)




In [ ]:
y_train

In [ ]:
X_train.head() 

In [ ]:
trainX = pd.DataFrame(X_train)
trainX[colonne_rechercher] = y_train

testX = pd.DataFrame(X_test)
testX[colonne_rechercher] = y_test

In [ ]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [ ]:
# send data to S3. SageMaker will take training data from s3
sk_prefix = "sagemaker/donneetraintestcarythma/"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [ ]:
testpath

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# importing the models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
### CREATION  DU  SCRIPT D'ENTRAINEMENT 

In [ ]:
%%writefile script.py


from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd

# inference functions ---------------

# def input_fn(request_body, request_content_type):
#     print(request_body)
#     print(request_content_type)
#     if request_content_type == "text/csv":
#         request_body = request_body.strip()
#         try:
#             df = pd.read_csv(StringIO(request_body), header=None)
#             return df
        
#         except Exception as e:
#             print(e)
#     else:
#         return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "modelcarythma.joblib"))
    return clf

# def predict_fn(input_data, model):
#     if type(input_data) != str:
#         prediction = model.predict(input_data)
#         print(prediction)
#         return prediction
#     else:
#         return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    colonne_num = list(train_df.columns)
    colonne_rechercher = colonne_num.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[colonne_num]
    X_test = test_df[colonne_num]
    y_train = train_df[colonne_rechercher]
    y_test = test_df[colonne_rechercher]

    print('Column order: ')
    print(colonne_num)
    print()
    
    print("Label column is: ",colonne_rechercher)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1) #HistGradientBoostingClassifier() #HistGradientBoostingClassifier(max_iter=100, random_state=42, verbose=3, n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

In [ ]:
! python script.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

In [ ]:
#### Train script in-side Sagemaker container.

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.t3.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [ ]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)